In [1]:
import cvxpy as cp
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import scipy.io
# mosek only needed if we don't use MW
#import mosek
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import cProfile
#from baselines import *
from scipy.linalg import sqrtm

In [2]:
#synthetic example with N datapoints, in d dimensions, with k actions, random regressors
#returns a dictionary of covariates x, labels y, regressors ell.  
def synthetic_example(N,d,k):
    #covariates
    x = np.random.rand(N,d)
    #regressors
    ell = np.random.rand(k,d)
    #labels
    y = np.zeros((N,k))
    for i in range(N):
        for j in range(k):
            feature = x[i,:]
            regressor = ell[j,:]
            y[i,j] = np.inner(feature,regressor)
    data = {'cov': x, 'label': y, 'reg': ell}
    return data

data = synthetic_example(1000,100,10)    
print(data)
        

{'cov': array([[0.70380728, 0.01411571, 0.440148  , ..., 0.04888691, 0.57769466,
        0.23135073],
       [0.21697098, 0.78522541, 0.42974641, ..., 0.27308731, 0.116627  ,
        0.4803269 ],
       [0.94690984, 0.35707631, 0.40102957, ..., 0.25477811, 0.72914031,
        0.76891778],
       ...,
       [0.0419411 , 0.75554647, 0.61379518, ..., 0.54849168, 0.40545699,
        0.25055964],
       [0.85917243, 0.13045235, 0.03031786, ..., 0.61909847, 0.15385964,
        0.85581847],
       [0.24073465, 0.13927862, 0.9709762 , ..., 0.96158984, 0.52566188,
        0.44892995]]), 'label': array([[21.6813295 , 22.2086819 , 21.3786482 , ..., 19.67851435,
        20.39861053, 22.90546331],
       [26.33630615, 24.46240175, 27.81110713, ..., 23.81758938,
        24.82118699, 27.18102664],
       [26.0468052 , 24.56862451, 27.62107869, ..., 22.00420652,
        24.76944816, 26.85239768],
       ...,
       [28.60099706, 26.53347597, 29.64072317, ..., 24.42360873,
        27.66063249, 28.1716

In [3]:
from mw2 import MW_no_alt_min, get_weights, altmin_step

# def test_offline(Xs_train,Xs_test,Ys_train,pre_Ys_test,ids_test,params):
#     n,d = Xs_train.shape
#     w = [0.]*d
#     a = [1.]*n
#     iso_Xs_train, Sig_sqrt = isotropic(Xs_train)
#     # print "not putting in isotropic"
#     for i in range(AM_steps):
#         if i == 0:
#             init = False
#         else:
#             init = False
#         w,a = altmin_step(iso_Xs_train,Ys_train,a,params,init=init)
#         print(clean_loss(Xs_test,pre_Ys_test,np.matmul(Sig_sqrt,w),ids_test))
#     final_w = np.matmul(Sig_sqrt,w)
#     return final_w, clean_loss(Xs_test,pre_Ys_test,final_w,ids_test)

def isotropic(Xs,fake=False):
    if fake:
        return Xs, np.eye(Xs.shape[1])
    Sig = np.matmul(Xs.T,Xs)
    Sig_sqrt = np.linalg.inv(sqrtm(Sig))
    new_Xs = np.matmul(Xs,Sig_sqrt)
    return new_Xs, Sig_sqrt

def scram(x,y,params):
    N,d = x.shape
    w = [0.]*d
    a = [1.]*N
    #iso_x, Sig_sqrt = isotropic(x)
    iso_x = x
    Sig_sqrt = np.eye(d)
    AM_steps = 10
    for i in range(AM_steps):
        print('AM step: ',i)
        w,a = altmin_step(iso_x,y,a,params,init=False)
    final_w = np.matmul(Sig_sqrt,w)    
    return final_w

#regression oracle 
def regression_oracle(x,y,mode='ols'):
    if mode == 'ols':
        model = LinearRegression()
        model.fit(x,y)
        regressor = model.coef_
    if mode == 'scram':
        lr = 0.5
        lam = 0.2
        MW_steps = 10
        eta = 0.1
        params = (lr,lam,MW_steps,eta)
        regressor = scram(x,y,params)
    return regressor

x = data['cov']
y = data['label']
ell = data['reg']

reg = regression_oracle(x,y[:,0])

#print(reg)
#print(ell)

In [4]:
#contextual bandits takes covariates and labels
def contextual_bandit(cov_label):
    cov = cov_label['cov']
    labels = cov_label['label']
    (N,d) = cov.shape
    (N,k) = labels.shape
    estimators = np.zeros((k,d))
    action_list = []
    mu = k
    delta = 0.1
    gamma = np.sqrt(k*N/(d*np.log(N/d) + 1./(2*delta)))
    params = (mu,gamma)
    for i in range(N):
        print('iteration: ',i)
        covariate = cov[i,:]
        values = np.zeros(k)
        for j in range(k):
            est = estimators[j,:]
            values[j] = np.inner(est,covariate)
        action = select_action(values,params)
        action_list.append(action)
        bandit_feedback = labels[i,action]
        (data_x,data_y) = get_data(cov_label,action_list,action) 
        
        #bug, ols can run on one datapoint but scram can't
        estimators[action,:] = regression_oracle(data_x,data_y,mode='scram')
        #print(action)
        #print('ESTIMATORS')
        #print(estimators)
    return estimators

def get_data(cov_label,action_list,action):
    cov = cov_label['cov']
    labels = cov_label['label']
    (N,d) = cov.shape
    (N,k) = labels.shape
    length = len(action_list)
    count = 0
    for i in range(length):
        if action_list[i] == action:
            count = count + 1
    
    data_x = np.zeros((count,d))
    data_y = np.zeros(count)
    counter = 0
    for i in range(len(action_list)):
        if action_list[i] == action:
            data_x[counter] = cov[i,:]
            data_y[counter] = labels[i,action]
            counter = counter + 1
    return (data_x,data_y) 


def select_action(values,params):
    (mu,gamma) = params
    k = mu
    min_value = np.amin(values)
    min_index = np.where(values == min_value)[0][0]
    prob = np.zeros(len(values))
    for i in range(k): 
        if i == min_index:
            next
        else: 
            prob[i] = 1./(mu + gamma*(values[i]-min_value))
    prob[min_index] = 1 - np.sum(prob)
    #TODO roulette wheel
    draw = np.random.rand()
    sums = 0
    for i in range(k):
        sums = sums + prob[i]
        if sums >= draw:
            action = i
            break
    return action

est = contextual_bandit(data)
#print('ground truth')
#print(ell)
#print('estimate')
#print(est)
print('frobenius norm')
print(np.linalg.norm(ell - est))

iteration:  0
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  1
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  2
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  3
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  4
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  5
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  6
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM ste

AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  59
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  60
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  61
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  62
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  63
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  64
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  65
AM step:  0
0

AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  113
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  114
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  115
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  116
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  117
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  118
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:

AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  168
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  169
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  170
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  171
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  172
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  173
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:

0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  224
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  225
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  226
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  227
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  228
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  229
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM ste

...

0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  769
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  770
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  771
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  772
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  773
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  774
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM ste

AM step:  8
0
AM step:  9
0
iteration:  824
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  825
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  826
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  827
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  828
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  829
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  830
AM step:  0
0
AM ste

0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  879
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  880
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  881
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  882
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  883
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  884
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM ste

AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  933
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  934
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  935
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  936
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  937
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  938
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:

AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  987
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  988
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  989
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  990
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  991
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step:  9
0
iteration:  992
AM step:  0
0
AM step:  1
0
AM step:  2
0
AM step:  3
0
AM step:  4
0
AM step:  5
0
AM step:  6
0
AM step:  7
0
AM step:  8
0
AM step: